In [13]:
#current_dir = pwd()
#app_dir = "/app"
#cd(app_dir)

#push!(LOAD_PATH, app_dir)
#push!(LOAD_PATH, current_dir)

using Gen
using Distributions
using LinearAlgebra
using Flux
using Random
using Distances
using JLD
using Serialization
using StatsBase

include("docker-parallel/NUTS.jl")
include("docker-parallel/RJNUTS.jl")
include("docker-parallel/utils.jl")
include("docker-parallel/proposals.jl")
include("docker-parallel/LoadData.jl");

In [8]:
println(Threads.nthreads())

4


In [14]:
#BNN for Jupyter Notebook - not needed for script

#-------------------
#Bayesian Neural Net
#-------------------
function G(x, trace)
    activation = relu
    l = trace[:l]
    ks = [trace[(:k,i)] for i=1:l]
    
    for i=1:l
        in_dim, out_dim = layer_unpacker(i, l, ks)
        W = reshape(trace[(:W,i)], out_dim, in_dim)
        b = reshape(trace[(:b,i)], trace[(:k,i)])
        nn = Dense(W, b, activation)
        x = nn(x)
    end
    
    Wₒ = reshape(trace[(:W,l+1)], 1, ks[l])
    bₒ = reshape(trace[(:b,l+1)], 1)
    
    nn_out = Dense(Wₒ, bₒ)
    return nn_out(x)
    
end;

#-------------------
#Probabilistic Model
#-------------------
@gen function interpolator(x)
    
    #Node hyperparameters
    k_real = 4 #Number of hidden nodes per layer
    k_vector = [0.0 for i=1:k_real]
    k_vector[k_real] = 1.0

    #Layer hyperparameters
    l_range = 8 #Maximum number of layers in the network
    l_list = [Int(i) for i in 1:l_range]
    
    #Hyperpriors
    αᵧ = 1 #Regression Noise Shape
    βᵧ = 1 #Regression Noise Scale/Rate
    α₁ = 1 #Input Weights, Biases Shape
    β₁ = 1 #Input Weights, Biases Scale/Rate
    α₂ = 1 #Hidden & Output Weights Shape
    β₂ = k_real; #Hidden & Output Weights Scale
    
    d = length(x[:,1])
    
    #Create a blank choicemap
    obs = choicemap()::ChoiceMap
    
    #Draw number of layers
    l ~ categorical([1/length(l_list) for i=1:length(l_list)])
    l_real = l
    obs[:l] = l
    
    #Create individual weight and bias vectors
    #Loop through hidden layers
    k = [Int(0) for i=1:l+1]
    for i=1:l
        k[i] = @trace(categorical(k_vector), (:k,i))
        obs[(:k,i)] = k[i]
    end
    k[l+1] = @trace(categorical([1.0]), (:k,l+1))
    obs[(:k,l+1)] = k[l+1]
    
    #####################################
    #New hyperparameter schedule - Mar 8#
    #####################################
    
    τ = [0.0 for i=1:l+1]
    τᵦ = [0.0 for i=1:l+1]
    σ = [0.0 for i=1:l+1]
    σᵦ = [0.0 for i=1:l+1]
    
    for i=1:l+1
        if i==1
            τ[i] = @trace(gamma(α₁,β₁), (:τ,i))
            τᵦ[i] = @trace(gamma(α₁,β₁), (:τᵦ,i))
        else
            τ[i] = @trace(gamma(α₂,β₂), (:τ,i))
            τᵦ[i] = @trace(gamma(α₁,β₁), (:τᵦ,i))
        end
        σ[i] = 1/τ[i]
        σᵦ[i] = 1/τᵦ[i]
    end
    
    #Noise Variance
    τᵧ ~ gamma(αᵧ,βᵧ)
    σᵧ = 1/τᵧ
    
    #Sample weight and bias vectors
    W = [zeros(k[i]) for i=1:l+1]
    b = [zeros(k[i]) for i=1:l+1]

    for i=1:l+1
        if i == 1
            h = Int(d * k[i])
        else
            h = Int(k[i-1] * k[i])
        end

        if i<=l
            #Hidden Weights
            u = zeros(h)
            S = Diagonal([σ[i] for j=1:length(u)])
            W[i] = @trace(mvnormal(u,S), (:W,i))
            obs[(:W,i)] = W[i]
            
            #Hidden Biases
            ub = zeros(k[i])
            Sb = Diagonal([σᵦ[i] for j=1:length(ub)])   
            b[i] = @trace(mvnormal(ub,Sb), (:b,i))
            obs[(:b,i)] = b[i]
        else
            #Output Weights
            u = zeros(k[l])
            S = Diagonal([σ[i] for j=1:length(u)])
            W[i] = @trace(mvnormal(u,S), (:W,i))
            obs[(:W,i)] = W[i]

            #Output Bias
            ub = zeros(1)
            Sb = Diagonal([σᵦ[i] for j=1:length(ub)])
            b[i] = @trace(mvnormal(ub,Sb), (:b,i))
            obs[(:b,i)] = b[i]
        end
    end
    
    #Return Network Scores for X
    scores = transpose(G(x,obs))[:,1]
    
    #Regression Likelihood
    Sy = Diagonal([σᵧ for i=1:length(x[1,:])])
    y = @trace(mvnormal(vec(scores), Sy), (:y))

    return scores
    
end;

In [26]:
println("Packages Loaded")

filename = "Parallel1.jld"
ITERS = 3
CHAINS = Threads.nthreads()

#---------------
#Hyperparameters
#---------------

#NUTS hyperparameters
Δ_max = 10
acc_prob = 0.65
m=1

#Network hyperparameters
k_real = 4 #Number of hidden nodes per layer
k_vector = [0.0 for i=1:k_real]
k_vector[k_real] = 1.0

#Layer hyperparameters
l_range = 8 #Maximum number of layers in the network
l_list = [Int(i) for i in 1:l_range]
l_real = 1;

#Hyperprior Hyperparameters
αᵧ = 1 #Regression Noise Shape
βᵧ = 1 #Regression Noise Scale/Rate
α₁ = 1 #Input Weights, Biases Shape
β₁ = 1 #Input Weights, Biases Scale/Rate
α₂ = 1 #Hidden & Output Weights Shape
β₂ = k_real; #Hidden & Output Weights Scale

obs_master = choicemap()::ChoiceMap
obs_master[:y] = y_train
obs = obs_master;

#-----------
#Parallelize
#-----------

starting_traces = [generate(interpolator, (x_train,), obs)[1] for i=1:CHAINS]
traces = [[] for i=1:CHAINS]

#-----------
#Initialize
#-----------

println("Initializing Trace")
println("------------------")

#(trace,) = generate(interpolator, (x_train,), obs)
#trace = find_best_trace(x_train,y_train,100)

#--------------
#Run Inference
#--------------
#cd(current_dir)
println("Beginning Inference")
println("-------------------")
#traces, scores = RJNUTS(trace, ITERS)

Threads.@threads for i=1:CHAINS
    #@inbounds println("Chain $i")
    #@inbounds println(starting_traces[i][:l])
    traces[i], _ = @inbounds RJNUTS(starting_traces[i], ITERS)
end

#serialize(filename, traces)

Packages Loaded
Initializing Trace
------------------
Beginning Inference
-------------------
1 : -1048.8339908789117
2 : -1056.3682405984691
********** Accepted: 7 **********
********** Accepted: 2 **********
3 : -925.733951915466
1 : -516.105670062725
1 : -696.254861726435
1 : -505.29560003693473
********** Accepted: 3 **********
2 : -513.3504271603647
2 : -664.0924996299485
********** Accepted: 2 **********
3 : -497.2928135935364
2 : -495.3358071526677
3 : -604.1319830734994
3 : -477.8369612362302


In [27]:
ls = [[traces[i][j][:l] for j=1:length(traces[i])] for i=1:CHAINS]

4-element Array{Array{Int64,1},1}:
 [1, 1, 2]
 [2, 3, 2]
 [3, 3, 3]
 [7, 7, 7]

In [32]:
serialize(filename, traces)

In [33]:
recovered = deserialize("Parallel1.jld");

In [35]:
recovered[1]

3-element Array{Any,1}:
 Gen.DynamicDSLTrace{DynamicDSLFunction{Any}}(DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], ##interpolator#494, Bool[0], false), Trie{Any,Gen.ChoiceOrCallRecord}(Dict{Any,Gen.ChoiceOrCallRecord}((:k, 2) => Gen.ChoiceOrCallRecord{Int64}(1, 0.0, NaN, true),(:τᵦ, 2) => Gen.ChoiceOrCallRecord{Float64}(0.6925640891928133, -0.6925640891928133, NaN, true),(:b, 2) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([0.12166031987103637], -1.1077411833957787, NaN, true),(:b, 1) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([0.1851313214943689, 0.35452812581265275, 0.19106436593474324, 1.0373018368270273], -4.26619887938738, NaN, true),:l => Gen.ChoiceOrCallRecord{Int64}(1, -2.0794415416798357, NaN, true),(:W, 2) => Gen.ChoiceOrCallRecord{Array{Float64,1}}([0.6183694176034724, -0.3674427060226869, -1.2165696624155498, -0.7902583042370694], -4.832230586863183, NaN, true),(:k, 1) => Gen.ChoiceOrCallRecord{Int64}(